In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

## Github Metrics for Stylus Sprint Projects
Query key GitHub activity metrics from 2024-2025 for Stylus Sprint projects, providing insights into project development velocity and community engagement.

In [2]:

query = """
    SELECT  p.display_name as Name,
            m.metric_name as Metric,
            ts.sample_date as Date,
            ts.amount as Value
    FROM metrics_v0 m
    JOIN timeseries_metrics_by_project_v0 ts
    on m.metric_id = ts.metric_id
    JOIN projects_v1 p
    on p.project_id = ts.project_id
    where metric_name like 'GITHUB_%'
    and YEAR(ts.sample_date) in (2024, 2025)
    and p.project_name IN (
        '9lives-fluidity-money',
        'angel',
        'arbos-foundry-bernard-wagner',
        'cobuilders-xyz',
        'codetracer-metacraft-labs',
        'ember-shahankhatch',
        'enclave-gnosisguild',
        'fairblock',
        'fiet-protocol-usherlabs',
        'nuffle-labs',
        'opensource-observer',
        'passportxyz',
        'pyth-network',
        'rather_labs',
        'redstone-finance',
        'remix-arbitrum-welldonestudio',
        'runtimeverification',
        'solang-hyperledger-solang',
        'stylusport-teamscanworks',
        'sylow-warlock-labs',
        'syndicateprotocol',
        'third-web',
        'trailofbits',
        'wakeuplabs',
        'walnuthq')
"""

df = client.to_pandas(query)
df['Date'] = pd.to_datetime(df['Date'])

df.to_csv('stylus_github_metrics.csv', index=False)
df

,Name,Metric,Date,Value
0,Trail of Bits Security Reviews,GITHUB_comments_daily,2024-01-08,3.0
1,Pyth Network,GITHUB_comments_daily,2024-01-08,19.0
2,Open Source Observer,GITHUB_comments_daily,2024-01-08,7.0
3,thirdweb,GITHUB_comments_daily,2024-01-08,30.0
4,Runtime Verification,GITHUB_comments_daily,2024-01-08,18.0
...,...,...,...,...
49779,Runtime Verification,GITHUB_opened_issues_daily,2024-05-30,3.0
49780,thirdweb,GITHUB_opened_issues_daily,2024-05-30,1.0
49781,Open Source Observer,GITHUB_opened_issues_daily,2024-05-30,3.0
49782,Open Source Observer,GITHUB_closed_issues_daily,2024-05-30,2.0


## Mapping the Stylus SDK Ecosystem

This analysis tracks the network of dependencies for projects using the Stylus SDK for Rust. By examining package relationships and ownership patterns, we can understand how the Stylus ecosystem is growing and evolving. 

In [3]:
query = """
WITH seed_repos AS (
  -- First get all repos that depend on stylus-sdk-rs packages
  SELECT DISTINCT 
    sboms.from_artifact_namespace,
    sboms.from_artifact_name
  FROM sboms_v0 sboms
  JOIN package_owners_v0 package_owners
    ON sboms.to_package_artifact_name = package_owners.package_artifact_name
    AND sboms.to_package_artifact_source = package_owners.package_artifact_source
  WHERE package_owners.package_owner_artifact_namespace = 'offchainlabs'
    AND package_owners.package_owner_artifact_name = 'stylus-sdk-rs'
)
SELECT DISTINCT
  sboms.from_artifact_namespace as seed_repo_owner,
  sboms.from_artifact_name as seed_repo_name,
  sboms.to_package_artifact_name as package_name,
  package_owners.package_owner_artifact_namespace as package_repo_owner,
  package_owners.package_owner_artifact_name as package_repo_name,
  sboms.to_package_artifact_source as package_source
FROM sboms_v0 sboms
JOIN package_owners_v0 package_owners
  ON sboms.to_package_artifact_name = package_owners.package_artifact_name
  AND sboms.to_package_artifact_source = package_owners.package_artifact_source
JOIN seed_repos
  ON sboms.from_artifact_namespace = seed_repos.from_artifact_namespace
  AND sboms.from_artifact_name = seed_repos.from_artifact_name
WHERE package_owners.package_owner_artifact_namespace IS NOT NULL
"""
df = client.to_pandas(query)

df.to_csv('stylus-sdk-rs-dependencies.csv', index=False)

df



,seed_repo_owner,seed_repo_name,package_name,package_repo_owner,package_repo_name,package_source
0,theblokc,arbdevbootcamp-reference,update-browserslist-db,browserslist,update-db,NPM
1,teeeeeo9,basket-swap-stylus,strum_macros,peternator7,strum,RUST
2,seabrick,seabrick-contracts,strum_macros,peternator7,strum,RUST
3,dsrvlabs,arbiturm-stylus-solidity-compare,strum_macros,peternator7,strum,RUST
4,offchainlabs,stylus-quickstart-vending-machine,strum_macros,peternator7,strum,RUST
...,...,...,...,...,...,...
14525,offchainlabs,stylus-erc721,string-width,sindresorhus,string-width,NPM
14526,theblokc,arbdevbootcamp-reference,string-width,sindresorhus,string-width,NPM
14527,offchainlabs,stylus-erc721,mock-fs,tschaub,mock-fs,NPM
14528,offchainlabs,stylus-erc721,eth-ens-namehash,danfinlay,eth-ens-namehash,NPM
